In [6]:
import heapq

class Node:
    def __init__(self, position, g_cost=0, h_cost=0):
        self.position = position
        self.g_cost = g_cost  # Cost from start to this node
        self.h_cost = h_cost  # Heuristic cost from this node to end
        self.f_cost = g_cost + h_cost  # Total cost (g_cost + h_cost)
        self.parent = None  # Parent node in the path

    def __lt__(self, other):
        return self.f_cost < other.f_cost

class AStar:
    def __init__(self, grid, start, end):
        self.grid = grid
        self.start = start
        self.end = end
        self.open_list = []
        self.closed_list = set()
        self.start_node = Node(start)
        self.end_node = Node(end)

    def heuristic(self, current_position):
        # Using Manhattan distance as heuristic
        return abs(current_position[0] - self.end[0]) + abs(current_position[1] - self.end[1])

    def get_neighbors(self, node):
        neighbors = []
        x, y = node.position
        directions = [(-1, 0), (1, 0), (0, -1), (0, 1)]  # 4 directions: up, down, left, right

        for dx, dy in directions:
            new_position = (x + dx, y + dy)
            if 0 <= new_position[0] < len(self.grid) and 0 <= new_position[1] < len(self.grid[0]):
                if self.grid[new_position[0]][new_position[1]] != 1:  # 1 represents an obstacle
                    neighbors.append(new_position)

        return neighbors

    def reconstruct_path(self, current_node):
        path = []
        while current_node is not None:
            path.append(current_node.position)
            current_node = current_node.parent
        return path[::-1]  # Reverse the path to get the correct order

    def find_path(self):
        heapq.heappush(self.open_list, self.start_node)

        while self.open_list:
            current_node = heapq.heappop(self.open_list)
            self.closed_list.add(current_node.position)

            # If we have reached the end
            if current_node.position == self.end:
                return self.reconstruct_path(current_node)

            neighbors = self.get_neighbors(current_node)
            for neighbor_position in neighbors:
                if neighbor_position in self.closed_list:
                    continue

                g_cost = current_node.g_cost + 1
                h_cost = self.heuristic(neighbor_position)
                neighbor_node = Node(neighbor_position, g_cost, h_cost)
                neighbor_node.parent = current_node

                if not any(neighbor_node.position == open_node.position and neighbor_node.f_cost >= open_node.f_cost
                           for open_node in self.open_list):
                    heapq.heappush(self.open_list, neighbor_node)

        return None  # No path found

# Function to take grid input from the user
def get_user_input():
    rows = int(input("Enter the number of rows in the grid: "))
    cols = int(input("Enter the number of columns in the grid: "))

    grid = []

    print("Enter the grid row by row (0 for free space, 1 for obstacle):")
    for i in range(rows):
        row = list(map(int, input(f"Enter row {i+1}: ").split()))
        grid.append(row)

    start = tuple(map(int, input("Enter the start position (row, col): ").split()))
    end = tuple(map(int, input("Enter the end position (row, col): ").split()))

    return grid, start, end

# Main execution
if __name__ == "__main__":
    grid, start, end = get_user_input()

    # Validate the start and end positions
    if grid[start[0]][start[1]] == 1 or grid[end[0]][end[1]] == 1:
        print("Start or end position is blocked. Please choose different positions.")
    else:
        astar = AStar(grid, start, end)
        path = astar.find_path()

        if path:
            print("Path found:", path)
        else:
            print("No path found.")


Enter the number of rows in the grid: 5
Enter the number of columns in the grid: 5
Enter the grid row by row (0 for free space, 1 for obstacle):
Enter row 1: 0 0 0 0 0
Enter row 2: 0 1 1 0 0
Enter row 3: 0 0 1 0 0
Enter row 4: 0 0 0 1 0
Enter row 5: 0 0 0 0 0
Enter the start position (row, col): 0 0
Enter the end position (row, col): 4 4
Path found: [(0, 0), (1, 0), (2, 0), (3, 0), (3, 1), (3, 2), (4, 2), (4, 3), (4, 4)]
